<a href="https://colab.research.google.com/github/Madhan-sukumar/CNN/blob/main/Image%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About Dataset:
- CIFAR-10 Photo Classification Dataset
- CIFAR is an acronym that stands for the Canadian Institute For Advanced Research and the CIFAR-10 dataset was developed along with the CIFAR-100 dataset by researchers at the CIFAR institute.

- The dataset is comprised of 60,000 32×32 pixel color photographs of objects from 10 classes, such as frogs, birds, cats, ships, etc. The class labels and their standard associated integer values are listed below.

- 0: airplane
1: automobile
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck

In [ ]:
# imorting dependencies
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Hyper Paramteres
num_epoch = 4
batch_size = 4
learning_rate = 0.001

In [ ]:
#dataset has PILImage images of range [0,1]
#we transform them to sensors and normalise between [-1,1]
# Normalize(mean, std[, inplace]) : Normalize a tensor image with mean and standard deviation.

transform = transforms.Compose(
                                  [transforms.ToTensor(),
                                  transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
                              )


In [ ]:
#downloading and splitting the dataset
# Importing the dataset from CIFAR and transforming

train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                           download=True, transform = transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                           download = True, transform = transform)

100%|██████████| 170498071/170498071 [00:04<00:00, 42297324.66it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Data Loaders
train_loader = DataLoader(dataset= train_dataset, batch_size = batch_size,shuffle = True)
test_loader = DataLoader(dataset= test_dataset, batch_size = batch_size,shuffle = False)

In [ ]:
#defining the classes
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

# IMPLEMENTING CONV NET

In [ ]:
class ConvNet(nn.Module):

  #creating conv layer and fully connected layer
  def __init__(self):
    super(ConvNet, self).__init__()
    #creating 1st conv layer,input has 3 channels, 6 - output as 6 channels, kernal or filter to slide over image -5 
    self.conv1 = nn.Conv2d(3,6,5)
    #max pooling layer 2x2
    self.pool = nn.MaxPool2d(2,2)
    #creating 2nd conv layer, input channel must equal to last output channel
    self.conv2 = nn.Conv2d(6,16,5)

    #fully connected 3 layers with input size and output size
    #16x5x5 - got output of 2nd conv layer as 16 as output channel and 5x5 after pooling
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10) # 10 for 10 different classes

  #defining forward, x will get the sample as input
  def forward(self,x):
    #conv1 --> relu --> pooling --> conv2 -- relu --> pooling --Flattening --> fully con layer
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    #flatening
    x= x.view(-1,16*5*5)
    #passing to fullycon layer1 --> applying relu function --> fullycon2 --> relu --> fullycon3 -- > output
    x= F.relu(self.fc1(x))
    x= F.relu(self.fc2(x))
    x= self.fc3(x)
    return x

model = ConvNet().to(device)

# LOSS AND OPTIMISER

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# TRAINING LOOP

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):  #i - index
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device) #pushing to GPU for GPU support
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/4], Step [2000/12500], Loss: 1.6754
Epoch [1/4], Step [4000/12500], Loss: 2.3011
Epoch [1/4], Step [6000/12500], Loss: 2.1244
Epoch [1/4], Step [8000/12500], Loss: 1.0895
Epoch [1/4], Step [10000/12500], Loss: 1.1142
Epoch [1/4], Step [12000/12500], Loss: 1.2080
Epoch [2/4], Step [2000/12500], Loss: 1.3731
Epoch [2/4], Step [4000/12500], Loss: 1.5377
Epoch [2/4], Step [6000/12500], Loss: 1.6718
Epoch [2/4], Step [8000/12500], Loss: 1.2515
Epoch [2/4], Step [10000/12500], Loss: 0.8482
Epoch [2/4], Step [12000/12500], Loss: 1.8774
Epoch [3/4], Step [2000/12500], Loss: 1.7664
Epoch [3/4], Step [4000/12500], Loss: 2.6744
Epoch [3/4], Step [6000/12500], Loss: 2.2780
Epoch [3/4], Step [8000/12500], Loss: 2.2778
Epoch [3/4], Step [10000/12500], Loss: 0.7260
Epoch [3/4], Step [12000/12500], Loss: 0.9087
Epoch [4/4], Step [2000/12500], Loss: 1.0786
Epoch [4/4], Step [4000/12500], Loss: 1.1327
Epoch [4/4], Step [6000/12500], Loss: 1.8083
Epoch [4/4], Step [8000/12500], Loss: 1.3081
Epoc

# TESTING 

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1) #1 for index 1
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 54.44 %
Accuracy of plane: 56.4 %
Accuracy of car: 75.0 %
Accuracy of bird: 33.6 %
Accuracy of cat: 23.8 %
Accuracy of deer: 44.9 %
Accuracy of dog: 52.9 %
Accuracy of frog: 68.1 %
Accuracy of horse: 64.8 %
Accuracy of ship: 68.2 %
Accuracy of truck: 56.7 %
